In [ ]:
USE_TPU = 0
USE_GPU = 1

assert (USE_TPU == 0) | (USE_GPU == 0)

In [ ]:
!pwd

/content


In [ ]:
%cd /content/drive/MyDrive/MentalHealthShared

/content/drive/MyDrive/MentalHealthShared


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 6.7 MB/s eta 0:00:00
  Created wheel for ratelimit: filename=ratelimit-2.2.1-py3-none-any.whl size=5909 sha256=58927577753287e870b3284c64c399f42a4a24cf22371894f9de79d7b9867ba2
  Stored in directory: /root/.cache/pip/wheels/8e/34/5e/1dd3d652594bdf5df01109a683a455121d0b726978051bf720
Successfully built ratelimit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [ ]:
ON_COLAB=False
if ON_COLAB:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    BASEDIR='/content/drive/MyDrive/MentalHealthShared'
    PYTHONDIR=BASEDIR+'src'
    RESULTSDIR=BASEDIR+'results/'
    MODELSDIR=BASEDIR+'model/'
    DATADIR=BASEDIR+'data/'
    import utils 
    from pre_processing import findLast, convert_utc, check_missing_threads
else:
    import os
    BASEDIR = os.getcwd() + "/"
    dirs = ["results","model","data"]
    for dirc in dirs:
        if dirc not in os.listdir(): 
            os.makedirs(os.path.join(BASEDIR,dirc))
    PYTHONDIR=BASEDIR+'src/'
    RESULTSDIR=BASEDIR+'results/'
    MODELSDIR=BASEDIR+'model/'
    DATADIR=BASEDIR+'data/'
    from src import utils
    from src.pre_processing import findLast, convert_utc, check_missing_threads

In [ ]:
if USE_TPU:
      !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
      !python pytorch-xla-env-setup.py #--version $VERSION

      # imports pytorch
      import torch
      # imports the torch_xla package
      import torch_xla
      import torch_xla.core.xla_model as xm

      device = xm.xla_device()
      torch_xla._XLAC._xla_set_default_device(str(device))

else:
      import torch
      device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
      print(device)

cuda


In [ ]:
import sys
sys.path.append(PYTHONDIR)
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import pickle
import time
import pdb
import importlib
import pprint
import re

import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
# loading DistilBERT to obtain embeddings
!pip install -q transformers
# Use DistilBERT para representar cada post e comentario
#from transformers import DistilBertModel, DistilBertTokenizer

#tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
#model = DistilBertModel.from_pretrained('bert-base-portuguese-cased')

from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads

model = AutoModelForPreTraining.from_pretrained('neuralmind/bert-base-portuguese-cased')
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)

model = model.to(device)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.3 MB/s eta 0:00:00


In [ ]:
# loading VADER for sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')

# initialize VADER
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from datetime import timedelta

def areIntervalsShort(post, seqlen_series):
  postid = post.name
  #print(postid)
  if postid in seqlen_series.index:
    seqlen = seqlen_series.loc[postid]
    #print(seqlen)
    # from post (created_utc[0]) to last comment (created_utc[seqlen])
    timestamps = np.array(post.created_utc[:seqlen])
    timedeltas = timestamps[1:] - timestamps[:-1]
    #print(timedeltas)
    #print(timedelta(days=1))
    return np.all(timedeltas < 86400)#.total_seconds())

def _subtreeHasValidBranches(grouped, root, branches, is_author=False, prev = []):
  prev = prev + [root]
  if not root in grouped.groups:
    if (is_author == True) and (len(prev)>2):
      branches.append(prev)
      return True
  else:
    comments       = grouped.get_group(root).comment_id.values
    is_post_author = grouped.get_group(root).is_post_author.values
    any_subtree_valid = np.any(
        [_subtreeHasValidBranches(grouped, comment, branches, is_author, prev) \
         for comment, is_author in zip(comments, is_post_author)]
        )
    if any_subtree_valid:
      return True
    else:
      if (is_author == True) and (len(prev)>2):
        branches.append(prev)
        return True

  return False


def getValidBranches(post):
  grouped = pd.DataFrame({'is_post_author':post.is_post_author[1:],'comment_id':post.comments,'parent_id':post.parent_id}).groupby('parent_id')
  #grouped = pd.DataFrame({'is_post_author':post.is_post_author[1:],'comment_id':post.comments,}).groupby('parent_id')

  # for name, group in grouped:
  #   print(group)
  branches = []
  if _subtreeHasValidBranches(grouped, post.name, branches):
    # map comments to indexes
    ind_branches = []
    # add 1 to indices since columns features, score and is_post_author start with post info 
    comment2ind = dict(zip(post.comments, range(1,len(post.comments)+1)))
    for bx, branch in enumerate(branches):
      # include post, i.e. index 0
      ind_branches.append([0]+[comment2ind[comment_id] for comment_id in branch[1:]])

    return ind_branches
  else:
    return None

In [ ]:
def create_subreddit_df(reddit, infile_extension, suffix=''):
    infile_extension = infile_extension.lstrip('.')

    print('Loading dicts...')
    with open(f'{DATADIR}{reddit}_post2data{suffix}.{infile_extension}','rb') as infile:
        post2data = pickle.load(infile)
    with open(f'{DATADIR}{reddit}_post2comments{suffix}.{infile_extension}','rb') as infile:
        post2comments = pickle.load(infile)
    with open(f'{DATADIR}{reddit}_comment2data{suffix}.{infile_extension}','rb') as infile:
        comment2data = pickle.load(infile)

    outpath=f'{DATADIR}{reddit}{suffix}_distilbert_filtered_posts.parquet'
    print(f'Dataframe will be saved in {outpath}')

    print('creating df from post2data...')
    df = pd.DataFrame(list(post2data.values()), index=post2data.keys())
    df = df.rename( columns={'selftext':'text'})
    print(1, len(df))
 

    #df = df.drop(columns='num_comments')

    # join with list of comments
    print('joining with post2comments...')
    tmp = pd.Series(post2comments,name='comments')
    df = df.join(tmp, how='inner')
    print(2, len(df))

    post_df = df

    print('creating df from comment2data...')
    df = pd.DataFrame(list(comment2data.values()), index=comment2data.keys())
    df = df.rename( columns={'body':'text'})
    # THE FOLLOWING 2 LINES MUST BE USED ONLY IF THE LINK_IDS HAVE A PREFIX
    
    #df["link_id"] = df["link_id"].apply(lambda x:x[3:])
    #df["parent_id"] = df["parent_id"].apply(lambda x:x[3:]) 
    print(3, len(df))


    print('remove comments that correspond to inexistent threads...')
    #post_ids = set(post_df.index)
    #print(post_df.comments)
    #print(df.head())
    #pdb.set_trace()
    lst = post_df.comments.values.tolist()
    flat_list = [item for sublist in lst for item in sublist]
    post_ids = pd.DataFrame(flat_list, columns = ['col'])
    df = df[df.link_id.index.isin(post_ids.col)]
    print(4, len(df))

    #link dos dois contenham o mesmo id do post e do comentario pra ver se o autor da thread escreveu o comentario
    print('compute is_post_author...')
    
    post_df2 = post_df.explode('comments')
    #print(df.loc[df.index == "dbv0p23"])
    #pdb.set_trace()
    df = post_df2.join(df, on = 'comments', how = 'left' , rsuffix='_post')
    
    #df = df.merge(post_df['author'], right_on=['index'], left_on=['comments'] , how="inner", suffixes=('_post'))
    df['is_post_author'] = (df.author == df.author_post)
    #print(df)
    df = df.drop('author_post', axis=1)
    post_df = post_df.filter(items=df.index, axis=0)
    
    comment_df = df

    print('remove  posts without text...')
    post_df = post_df.dropna()
    # post_df.num_comments = post_df.num_comments.astype(int)
    print(5, len(post_df))

    print('remove posts with less than 2 comments...')
    # remove posts with less than 2 comments
    post_df['num_comments'] = post_df['comments'].apply(len) # use this as num_comments
    post_df = post_df[post_df['num_comments'] >= 2]
    print(6, len(post_df))

    print('remove posts with small text...')
    post_df = post_df[post_df['text'].apply(len) >= 2]
    print(7, len(post_df))

    print('remove removed or deleted posts...')
    post_df = post_df[(post_df['text'] != '[removed]') & (post_df['text'] != '[deleted]')]
    print(8, len(post_df))


    print('remove deleted authors...')
    post_df = post_df[(post_df['author'] != '[deleted]')]
    print(9, len(post_df))

    #pdb.set_trace()
    
    print('calculate sequence length when there are other commenters followed by post author...')
    #pdb.set_trace()
    comment_df['post_id'] = comment_df.index
    comment_df.set_index('comments', inplace = True)
    check_missing_threads(post_df,comment_df)
    temp = post_df.apply(lambda x: utils.getThreadLen(x, comment_df),axis=1)
    #post_df['seq_len']

    print('remove invalid threads...')
    post_df = post_df.dropna()
    print(10, len(post_df))

    print('remove comments without body...')
    #print(comment_df.loc[comment_df.index == "dbv0p23"])
    #comment_df = comment_df.dropna()
    print(11,len(comment_df))

    print('remove comments that correspond to deleted threads...')
    #pdb.set_trace()
    post_ids = set(post_df.index)
    comment_df = comment_df[comment_df.post_id.isin(post_ids)]
    print(12,len(comment_df))

    print('computing score...')
    post_df['score']    =    post_df['text'].apply(lambda x: utils.calculateParagraphScore(x,sid))
    comment_df['score'] = comment_df['text'].apply(lambda x: utils.calculateParagraphScore(x,sid))

    print("remove posts whose scores and posts whose comments' scores couldn't be computed...")
    posts_to_remove = list(post_df[post_df.score.isna()].index)
    posts_to_remove += list(comment_df[comment_df.score.isna()]['link_id'].values)
    post_df = post_df.drop(posts_to_remove)
    post_df = post_df.drop_duplicates(subset="created_utc", keep='first')
    print(13,len(post_df))

    print('remove comments whose posts were removed...')
    comment_df = comment_df[~comment_df.link_id.isin(posts_to_remove)]
    print(14,len(comment_df))

    print('concatenating post and comment columns...')
    post_df['parent_id'] = post_df.apply(lambda x: [comment_df.loc[cid, 'parent_id'] for cid in x.comments], axis=1)
    post_df['is_post_author'] = post_df.apply(lambda x: [True] + [comment_df.loc[cid, 'is_post_author'] for cid in x.comments], axis=1)
    post_df.score = post_df.apply(lambda x: [x.score] + [comment_df.loc[cid, 'score'] for cid in x.comments], axis=1)
    # post_df.score_noavg = post_df.apply(lambda x: [x.score_noavg] + [comment_df.loc[cid, 'score_noavg'] for cid in x.comments], axis=1)
    post_df.created_utc = post_df.apply(lambda x: [x.created_utc] + [comment_df.loc[cid, 'created_utc_post'] for cid in x.comments], axis=1)
    #print(post_df)

    # # ATTENTION! new seq_len includes post and last_comment
    # # for instance sequence [p, c0, c1, c2, ..., cn]
    # # where cn is the last comment made by author has seq_len=n+2

    # compute new seq_len:
    print('computing seq_len...')
    post_df['seq_len'] = post_df.is_post_author.apply(findLast)+1
    post_df['seq_len'] = post_df.seq_len.astype(int)

    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! -> linha 295
    print('computing DistilBERT features...')
    post_df['features'] = utils.extractFeatures(post_df, tokenizer, model, device, batch_size = 512)
    comment_df['features'] = utils.extractFeatures(comment_df, tokenizer, model, device, batch_size = 128)

    print('concatenating post and comment features...')
    post_df.features = post_df.apply(lambda x: [x.features] + [comment_df.loc[cid, 'features'] for cid in x.comments], axis=1)

    print('computing additional features for pre-filtering...')
    seqlen_series = utils.getNonOverlappingThreads(post_df)
    #   post_df = post_df.apply(lambda x: convert_utc(x),axis=1)
    are_intervals_short = post_df.apply(lambda x: areIntervalsShort(x,seqlen_series), axis=1)
    are_intervals_short = are_intervals_short.dropna()
    

    #seqlen_series = seqlen_series[(seqlen_series>=2) & are_intervals_short]
    seqlen_series = seqlen_series[(seqlen_series>=2)]
    seqlen_series.name = 'filtered_seqlen'
    post_df = post_df.join(seqlen_series)
    
    #print(comment_df)
    #print(post_df)

    validbranches_series = post_df.apply(getValidBranches,axis=1)
    validbranches_series.name = 'valid_branches'
    post_df = post_df.join(validbranches_series)

    columns_to_drop = [column for column in ['pos','neg','neu','compound','num_comments','score_noavg'] if column in post_df.columns]
    if columns_to_drop:
        print('Dropping superfluous columns...')
        post_df = post_df.drop(columns=columns_to_drop)

    #pdb.set_trace()
    post_df.to_parquet(outpath)

    return post_df 

In [ ]:
import pdb
from tqdm import tqdm
SUBREDDITS = ["AnsiedadeDepressao"]
for subreddit in SUBREDDITS:
    create_subreddit_df(subreddit,'pkl')

Loading dicts...


FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Experiments on the effect of ettiquete words
THE NEXT SECTION IS DEVOTED TO THE EXPERIMENTS DONE IN REGARDS TO THE EFFECT THAT ETTIQUETE WORDS HAVE ON VADER. NOT NECESSARY FOR THE MAIN MODEL

In [ ]:
RUN_ETTIQUETE_EXPS = False

In [ ]:
def filter_etiquette(subreddits,return_filtered_subreddits=False,etiquette_list = ["Thank You","Thank you","Thanks","Thx","Thankyou", "thank you", "thanks", "thx", "thankyou"],file_type = '_comment2data',file_extension='p'):
    if(type(subreddits) != type(list())):
        subreddits = [subreddits]
    print("Initializing...")
    total_perc = []
    filtered_subs = {}
    for subreddit in subreddits:
        print(f"Getting r/{subreddit} data...")
        with open(f'{DATADIR}no_thx_2017/{subreddit}_{file_type}.{file_extension}','rb') as infile:
            curr_sub_comments = pickle.load(infile)
        num_comments_with_etiquette = 0
        total_comments = len(curr_sub_comments)
        for key,comment in tqdm(curr_sub_comments.items()):
            for etiquette_word in etiquette_list:
                if etiquette_word in comment['text']:
                    num_comments_with_etiquette += 1
                    curr_sub_comments[str(key)]['text'] = comment['text'].replace(etiquette_word,"")
            if re.search(r"\b" + re.escape("ty") + r"\b",comment['text']):
                num_comments_with_etiquette += 1
                curr_sub_comments[str(key)]['text'] = re.sub(r"\b" + re.escape("ty") + r"\b","",comment['text'])
            if re.search(r"\b" + re.escape("Ty") + r"\b",comment['text']):
                num_comments_with_etiquette += 1
                curr_sub_comments[str(key)]['text'] = re.sub(r"\b" + re.escape("Ty") + r"\b","",comment['text'])
        
        print(f"Percentage of posts or comments filtered in r/{subreddit}: {(num_comments_with_etiquette/total_comments) * 100}%")
        total_perc.append([num_comments_with_etiquette,total_comments])
        print("Saving changes...")
        #pickle.dump(curr_sub_comments,open(DATADIR+ "no_thx_2017/" + f'{subreddit}_{file_type}.{file_extension}','wb'))
        filtered_subs[subreddit] = curr_sub_comments
        print(f"Done with r/{subreddit}")
    print(f"Total Percentage of posts or comments with etiquette words: {sum([total_perc[x][0] for x in range(len(total_perc))])/sum([total_perc[x][1] for x in range(len(total_perc))]) * 100}%")
    print("Done!")
    if(return_filtered_subreddits):
        return filtered_subs

In [ ]:
if RUN_ETTIQUETE_EXPS:
    subreddits = ['Anxiety','bipolar','depression','SuicideWatch']
    filter_etiquette(subreddits,file_type="comment2data",return_filtered_subreddits=True)